In [1]:
import pandas as pd
from sklearn import preprocessing
import numpy as np
from collections import defaultdict
import os
import pickle

from code.algorithm_alpha import algorithm_alpha
from code.algorithm_t import algorithm_t

sim = True

In [2]:
def compute_order(knots):
    if knots.ndim > 1:
        return np.flip(np.argsort(knots[:, 0]))
    else:
        return np.flip(np.argsort(knots))


def compute_out_groups(group):  # number of identified clusters
    groups_out = len(np.unique(group))
    return groups_out


algorithm_map = {
    'alpha': algorithm_alpha,
    't': algorithm_t
}


def check_algorithm(ran_var, ran_int, n_fix, sim, a_type, tol, model, n):
    all_knots = []
    all_par = []
    all_W = []
    all_hess_ran = []
    all_hess_fix = []
    all_group = []
    all_out_groups = []

    i = 0
    while i < n:
        try:
            print(i)
            knots, par, W, hess_ran, hess_fix = algorithm_map[a_type](ran_var, ran_int, n_fix, sim, tol, model, 
                                                                      fix=None, lengths=None, y=None, N=None, t=None)

            x = compute_order(knots)
            knots = knots[x]
            W = W[:, x]
            group = np.array([np.nan if np.sum(W[j, :]) == 0 else np.argmax(W[j, :]) for j in range(10)])
            out_groups = compute_out_groups(group)

            all_knots.append(knots)
            all_par.append(par)
            all_W.append(W)
            all_hess_ran.append(hess_ran)
            all_hess_fix.append(hess_fix)
            all_group.append(group)
            all_out_groups.append(out_groups)

            i = i + 1
        except:
            print('an error occurred')
            pass

    # save results
    # file_name = str(ran_var) + '_' + str(ran_int) + '_' + str(n_fix) + '_' + a_type + '_' + str(tol) + '_' + model + '_' + str(
    #    n) + '.pickle'
    # f = open('output/' + file_name, 'wb')
    # pickle.dump([all_knots, all_par, all_W, all_hess_ran, all_hess_fix, all_group, all_out_groups], f)
    # f.close()

    return all_knots, all_par, all_W, all_hess_ran, all_hess_fix, all_group, all_out_groups

In [3]:
ran_var = False
ran_int = True
n_fix = 1
a_type = 'alpha' # set 't' or 'alpha'
tol = 0.05
model = 'B'  # set 'B' or 'P'
n = 1

all_knots, all_par, all_W, all_hess_ran, all_hess_fix, all_group, all_out_groups = \
    check_algorithm(ran_var, ran_int, n_fix, sim, a_type, tol, model, n)

# save results
file_name = str(ran_var) + '_' + str(ran_int) + '_' + str(n_fix) + '_' + a_type + '_' + str(
    tol) + '_' + model + '_' + str(n) + '.pickle'
f = open('output/simulation_study_results/' + file_name, 'wb')
pickle.dump([all_knots, all_par, all_W, all_hess_ran, all_hess_fix, all_group, all_out_groups], f)
f.close()

0
[-11.6588    4.57888 -10.41022   5.043     2.40746   2.40675   5.043
   2.40624   5.04298 -11.72669]
D_triu
[[        nan 1.62377e+01 1.24858e+00 1.67018e+01 1.40663e+01 1.40655e+01
  1.67018e+01 1.40650e+01 1.67018e+01 6.78938e-02]
 [        nan         nan 1.49891e+01 4.64112e-01 2.17142e+00 2.17214e+00
  4.64118e-01 2.17264e+00 4.64095e-01 1.63056e+01]
 [        nan         nan         nan 1.54532e+01 1.28177e+01 1.28170e+01
  1.54532e+01 1.28165e+01 1.54532e+01 1.31647e+00]
 [        nan         nan         nan         nan 2.63554e+00 2.63625e+00
  5.76349e-06 2.63675e+00 1.76331e-05 1.67697e+01]
 [        nan         nan         nan         nan         nan 7.10702e-04
  2.63554e+00 1.21456e-03 2.63552e+00 1.41342e+01]
 [        nan         nan         nan         nan         nan         nan
  2.63625e+00 5.03860e-04 2.63623e+00 1.41334e+01]
 [        nan         nan         nan         nan         nan         nan
          nan 2.63676e+00 2.33966e-05 1.67697e+01]
 [        nan  

In [4]:
all_knots

[array([  4.5582 ,   2.39497, -10.88065])]

In [5]:
all_par

[array([-6.00342])]